In [15]:
import pandas as pd
import athena_utils as au
import locale
locale.setlocale(locale.LC_TIME, 'Spanish_Spain.1252')  # Windows



'Spanish_Spain.1252'

In [16]:
## PARAMETROS -------------------------------
proyecto = 72
convenio = "54-54-00-00"
hoy = pd.Timestamp.now().normalize()
fecha_inicio = '2025-01-01'
fecha_fin = '2025-12-30'
## -----------------------------------------

In [17]:
query_alumnos = fr"""
 
WITH 
-- ============================================== 
-- filtros de fecha 
-- ============================================== 
parametros AS ( 
    SELECT 
        DATE '2021-01-01' AS fecha_inicio, --just in case 
        CURRENT_DATE AS fecha_fin -- ídem 
)
-- ============================================== 
-- Alumnos por salon
-- ============================================== 
    SELECT distinct
        p.id as id,
		g.institution as institucion,
		g.grade as grado,
        c.id as room_id,
		g.student_id
    FROM datalake.room_roomsessions s 
    CROSS JOIN parametros 
    JOIN datalake.room_room_students b ON s.room_id = b.room_id 
    JOIN datalake.student_student f ON b.student_id = f.id 
    LEFT JOIN datalake.room_room c ON s.room_id = c.id 
    LEFT JOIN datalake.learning_group d ON c.group_id = d.id 
    LEFT JOIN datalake.learning_course e ON d.course_id = e.id 
    LEFT JOIN datalake.enrollment_enrolment g ON f.id = g.student_id AND e.id = g.course_id AND d.id = g.group_id 
    LEFT JOIN datalake.projects p ON g.b2b_project_id = p.id 
    LEFT JOIN datalake.attendance_attendance aa ON aa.room_session_id = s.id AND b.student_id = aa.object_id AND aa.content_type_id = 8 
    WHERE s.start_date BETWEEN parametros.fecha_inicio AND parametros.fecha_fin 
        AND COALESCE(g.state, 'active') NOT IN ('inactive', 'abandoned') 
        AND ( 
            (s.state = 'true' AND aa.status IN (1,2,3,4,5,6)) 
            OR (s.state = 'false' AND s.cancellation_reason_id IN (34,35)) 
        )
		and p.id in ({proyecto})
		

        """

In [18]:
query_cancelaciones = fr"""

WITH
cte_Organization AS (
    SELECT
        poa.project_id,
        ARRAY_JOIN(ARRAY_AGG(DISTINCT o.name), ', ') AS org_names
    FROM datalake.project_organization_association poa
    JOIN datalake.organizations o ON poa.organization_id = o.id
    GROUP BY poa.project_id
),
cte_ProgramType AS (
    SELECT
        ppta.project_id,
        ARRAY_JOIN(ARRAY_AGG(DISTINCT cbpt.name), ', ') AS program_types
    FROM datalake.project_program_type_association ppta
    JOIN datalake.catalog_b2bprogramtype cbpt ON ppta.program_type_id = cbpt.id
    GROUP BY ppta.project_id
),
base AS (
    SELECT
        DISTINCT
        p.id AS projectsID,
        p.name AS "Proyecto",
        p.internal_name,
        (CASE WHEN ee.institution IS NULL THEN ei.name ELSE ee.institution END) AS institucion,
        ei.municipality as Municipio,
        rr.id AS room,
        rs.id AS sesionID,
        rs.session_number AS sesion,
        ee.grade AS grado,
        (CASE WHEN rs.cancellation_reason_id IS NULL THEN 36 ELSE rs.cancellation_reason_id END) AS reasonID,
        (CASE WHEN rc.name IS NULL THEN 'N/A' ELSE rc.name END) AS Motivo,
        rs.start_date AS "Fecha",
        rs.state AS state,
        -- Profesor IED
        CONCAT(rraux.name, ' ', rraux.last_name) AS profesor_ied_nombre,
        CASE 
            WHEN aa_ied.status IN (1, 5) THEN true
            ELSE false
        END AS profesor_ied_asistio

    FROM
        datalake.room_roomsessions rs
        LEFT JOIN datalake.room_room rr ON rs.room_id = rr.id
        JOIN datalake.enrollment_enrolment ee ON rr.id = ee.room_id AND ee.b2b_project_id IS NOT NULL
        LEFT JOIN datalake.projects p ON (ee.b2b_project_id = p.id OR p.id = rr.project_b2b_id)
        LEFT JOIN cte_Organization o ON p.id = o.project_id
        LEFT JOIN cte_ProgramType pt ON p.id = pt.project_id
        LEFT JOIN datalake.account_user au ON rr.teacher_id = au.id
        LEFT JOIN datalake.catalog_reasonsessioncancellation rc ON (
            (CASE WHEN (rs.state = 'false' AND rs.cancellation_reason_id IS NULL) THEN 36 ELSE rs.cancellation_reason_id END) = rc.id
        )
        LEFT JOIN datalake.educational_institution ei ON ei.id = rr.educational_institution_id
        -- Join para asistencia del profesor IED
        LEFT JOIN datalake.attendance_attendance aa_ied 
            ON aa_ied.room_session_id = rs.id 
            AND aa_ied.room_id = rr.id 
            AND aa_ied.content_type_id = 276
        LEFT JOIN datalake.room_roomauxiliarteacher rraux 
            ON rraux.id = aa_ied.object_id

    WHERE p.id = {proyecto}
)
SELECT *
FROM base
WHERE
    NOT (state = 'false' AND projectsID IN (19, 14) AND (("Fecha" BETWEEN DATE '2024-06-17' AND DATE '2024-07-05') OR ("Fecha" BETWEEN DATE '2024-10-07' AND DATE '2024-10-14')))
    AND
    NOT (state = 'false' AND projectsID IN (47, 48, 56) AND ("Fecha" BETWEEN DATE '2024-10-07' AND DATE '2024-10-14'))
    AND
    (reasonID NOT IN (33, 34, 35) OR reasonID IS NULL)
    AND "Fecha" BETWEEN DATE '{fecha_inicio}' AND DATE '{fecha_fin}';

"""



In [19]:
df_cancelaciones = au.run_athena_query(query_cancelaciones)
df_estudiantes = au.run_athena_query(query_alumnos)

In [20]:
# convertir fecha a datetime y extraer mes-año
df_cancelaciones['fecha'] = pd.to_datetime(df_cancelaciones['fecha'])
df_cancelaciones['mes_año'] = df_cancelaciones['fecha'].dt.strftime('%B de %Y')

In [21]:
# Quedarme con el mes_año con 80% de frecuencia, si no, concatenar los 2 más frecuentes ordenados por fecha
mes_ano_counts = df_cancelaciones['mes_año'].value_counts()
total_count = mes_ano_counts.sum()
max_count = mes_ano_counts.max()

if max_count / total_count >= 0.8:
    mes_ano = mes_ano_counts.index[0]
else:
    top_two = mes_ano_counts.head(2)
    # ordenar top_two por fecha
    top_two = top_two.sort_index(key=lambda x: pd.to_datetime(x, format='%B de %Y'))
    mes_ano = f"{top_two.index[0]} y {top_two.index[1]}"

In [46]:
## Metricas por institucion ## 
dia_de_hoy = hoy.strftime('%d de %B de %Y')

# Obtener lista de instituciones únicas
instituciones = df_cancelaciones['institucion'].dropna().unique()

# Diccionario para almacenar métricas por institución
metricas_por_institucion = {}

for institucion in instituciones:
    # Filtrar datos por institución
    df_cancel_inst = df_cancelaciones[df_cancelaciones['institucion'] == institucion]
    df_est_inst = df_estudiantes[df_estudiantes['institucion'] == institucion]
    
    # Calcular métricas para esta institución
    sesiones_programadas = df_cancel_inst['sesionid'].nunique()
    sesiones_ejecutadas = df_cancel_inst[df_cancel_inst['state'] == 'true']['sesionid'].nunique()
    sesiones_dictadas = round(sesiones_ejecutadas / sesiones_programadas, 3) if sesiones_programadas > 0 else 0
    sesiones_asistidas = df_cancel_inst[
        (df_cancel_inst['state'] == 'true') & 
        (df_cancel_inst['profesor_ied_asistio'] == True)
    ]['sesionid'].nunique()
    porcentaje_asistencia = round(sesiones_asistidas / sesiones_ejecutadas, 3) if sesiones_ejecutadas > 0 else 0
    
    estudiantes = df_est_inst['student_id'].nunique()
    cantidad_salones = df_est_inst['room_id'].nunique()
    # lista de grados
    grados = sorted(df_est_inst['grado'].astype(int).unique())
    grados = ', '.join(f"{g}°" for g in grados[:-1]) + f" y {grados[-1]}°"

    # Guardar métricas en el diccionario
    metricas_por_institucion[institucion] = {
        'sesiones_programadas': sesiones_programadas,
        'sesiones_ejecutadas': sesiones_ejecutadas,
        'sesiones_dictadas': sesiones_dictadas,
        'sesiones_asistidas': sesiones_asistidas,
        'porcentaje_asistencia': porcentaje_asistencia,
        'estudiantes': estudiantes,
        'cantidad_salones': cantidad_salones,
        'grados': grados
    }

# Convertir a DataFrame para visualización
df_metricas = pd.DataFrame(metricas_por_institucion).T
df_metricas.index.name = 'institucion'
df_metricas

,sesiones_programadas,sesiones_ejecutadas,sesiones_dictadas,sesiones_asistidas,porcentaje_asistencia,estudiantes,cantidad_salones,grados
institucion,,,,,,,,
I.E.D. Marie Poussepin,337,288,0.855,272,0.944,368,12,"9°, 10° y 11°"
I.E.D. Villas De San Pablo,457,345,0.755,322,0.933,563,16,"9°, 10° y 11°"
E.N.S. La Hacienda,703,576,0.819,540,0.938,860,25,"9°, 10° y 11°"
I.E.D. San Vicente,174,160,0.92,157,0.981,205,6,"9°, 10° y 11°"
I.E.D. Alexander Von Humboldt,174,137,0.787,135,0.985,164,6,"9°, 10° y 11°"
I.E.D. El Campito,358,278,0.777,266,0.957,218,6,"9°, 10° y 11°"
I.E.D. Tierra Santa,86,73,0.849,71,0.973,75,3,"9°, 10° y 11°"
I.E.D. Evardo Turizo,286,264,0.923,253,0.958,347,10,"9°, 10° y 11°"
E.N.S. Del Distrito De Barranquilla,422,354,0.839,333,0.941,491,15,"9°, 10° y 11°"


In [47]:
from docx import Document
import re
import os

plantilla_path = 'plantilla.docx'

def extraer_texto_completo(doc):
    """Extrae todo el texto del documento para verificar variables"""
    texto = []
    
    # Párrafos
    for paragraph in doc.paragraphs:
        texto.append(paragraph.text)
    
    # Tablas
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                for paragraph in cell.paragraphs:
                    texto.append(paragraph.text)
    
    # Headers y footers
    for section in doc.sections:
        if section.header:
            for paragraph in section.header.paragraphs:
                texto.append(paragraph.text)
        if section.footer:
            for paragraph in section.footer.paragraphs:
                texto.append(paragraph.text)
    
    return '\n'.join(texto)

# Cargar plantilla y extraer texto
doc = Document(plantilla_path)
texto_completo = extraer_texto_completo(doc)

# Buscar todas las variables con formato <<variable>>
variables_encontradas = re.findall(r'<<([^>]+)>>', texto_completo)
print("Variables encontradas en la plantilla:")
for var in sorted(set(variables_encontradas)):
    print(f"  - <<{var}>>")

Variables encontradas en la plantilla:
  - <<Nombre de Docente>>
  - <<cantidad_estudiantes>>
  - <<cantidad_salones>>
  - <<dia_mes_ano>>
  - <<grados>>
  - <<ied>>
  - <<mes_ano>>
  - <<porcentaje_asistencia>>
  - <<sesiones_asistidas>>
  - <<sesiones_dictadas>>
  - <<sesiones_ejecutadas>>
  - <<sesiones_programadas>>


In [48]:
# Crear carpeta de salida
output_folder = 'actas_generadas'
os.makedirs(output_folder, exist_ok=True)

def reemplazar_en_parrafo(paragraph, variables):
    """Reemplaza variables manejando runs fragmentados"""
    texto_completo = paragraph.text
    
    # Verificar si hay alguna variable en el párrafo
    tiene_variable = any(f'<<{key}>>' in texto_completo for key in variables.keys())
    
    if tiene_variable:
        # Reemplazar todas las variables
        for key, value in variables.items():
            texto_completo = texto_completo.replace(f'<<{key}>>', str(value))
        
        # Limpiar todos los runs excepto el primero
        if paragraph.runs:
            paragraph.runs[0].text = texto_completo
            for run in paragraph.runs[1:]:
                run.text = ''

def reemplazar_variables_docx(doc, variables):
    """Reemplaza variables en todo el documento"""
    # Párrafos del cuerpo
    for paragraph in doc.paragraphs:
        reemplazar_en_parrafo(paragraph, variables)
    
    # Tablas
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                for paragraph in cell.paragraphs:
                    reemplazar_en_parrafo(paragraph, variables)
    
    # Headers y footers
    for section in doc.sections:
        if section.header:
            for paragraph in section.header.paragraphs:
                reemplazar_en_parrafo(paragraph, variables)
        if section.footer:
            for paragraph in section.footer.paragraphs:
                reemplazar_en_parrafo(paragraph, variables)
    
    return doc

# Generar un documento por cada institución
for institucion, metricas in metricas_por_institucion.items():
    doc = Document(plantilla_path)
    
    # Diccionario de variables a reemplazar
    variables = {
        'ied': institucion,
        'sesiones_programadas': metricas['sesiones_programadas'],
        'sesiones_ejecutadas': metricas['sesiones_ejecutadas'],
        'sesiones_dictadas': f"{metricas['sesiones_dictadas']*100:.1f}%",
        'sesiones_asistidas': metricas['sesiones_asistidas'],
        'porcentaje_asistencia': f"{metricas['porcentaje_asistencia']*100:.1f}%",
        'cantidad_estudiantes': metricas['estudiantes'],
        'cantidad_salones': metricas['cantidad_salones'],
        'dia_de_hoy': dia_de_hoy,
        'mes_ano': mes_ano,
        'convenio': convenio,
        'grados': metricas['grados']
    }
    
    doc = reemplazar_variables_docx(doc, variables)
    
    nombre_archivo = f"{institucion.replace('/', '-').replace('\\', '-')}.docx"
    doc.save(os.path.join(output_folder, nombre_archivo))
    print(f"Generado: {nombre_archivo}")

print(f"\nSe generaron {len(metricas_por_institucion)} documentos en '{output_folder}'")

Generado: I.E.D. Marie Poussepin.docx
Generado: I.E.D. Villas De San Pablo.docx
Generado: E.N.S. La Hacienda.docx
Generado: I.E.D. San Vicente.docx
Generado: I.E.D. Alexander Von Humboldt.docx
Generado: I.E.D. El Campito.docx
Generado: I.E.D. Tierra Santa.docx
Generado: I.E.D. Evardo Turizo.docx
Generado: E.N.S. Del Distrito De Barranquilla.docx
Generado: I.E.D. Fundación Pies Descalzos.docx
Generado: I.E.D. San Salvador.docx
Generado: I.E.D. Comunitaria Metropolitana.docx
Generado: I.E.D. Hogar Mariano.docx
Generado: I.E.D. Betania Norte.docx
Generado: I.E.D. Manuel Zapata.docx
Generado: I.E.D. Reuven Feuerstein.docx
Generado: I.E.D. Cruzada Social.docx
Generado: I.E.D. María Inmaculada.docx
Generado: I.E.D. Nueva Granada.docx
Generado: I.E.D. María Cano.docx
Generado: I.E.D. Meira Del Mar.docx
Generado: I.E.D. Juan José Rondón.docx
Generado: I.E.D. Ntra Señora Del Rosario.docx
Generado: I.E.D. Técnico De Comercio.docx
Generado: I.E.D. Brisas Del Río.docx
Generado: I.E.D. Las Flores.d